In [1]:
import os
import time
from samgeo import SamGeo
import json
import traceback

import ee
import geemap
import math
import sys, os
import json
from GEE_Func.S1_distor_dedicated import load_S1collection, S1_CalDistor, DEM_caculator
from GEE_Func.GEE_DataIOTrans import DataTrans, DataIO, Vector_process
from GEE_Func.GEE_CorreterAndFilters import ImageFilter, S1Corrector
from GEE_Func.GEE_Tools import S1_Cheker
from functools import partial
import traceback
from osgeo import gdal
Eq_pixels = DataTrans.Eq_pixels
from PackageDeepLearn.utils import DataIOTrans
import numpy as np

geemap.set_proxy(port=10809)
ee.Initialize()




# 初始化 SamGeo
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    device='cuda',
    sam_kwargs=None,
)

years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
SETP_Season = ['-02-25', '-05-31', '-09-15', '-11-28', '-02-25']

SaveDir = r'D:\Dataset_and_Demo'
SAM_PreDir = r'D:\Dataset_and_Demo\Processed'


## 删除不符合要求的_Trans.tif

In [4]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        
        # 存在任意则保留
        Cal_list = [i for i in Cal_list if (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
        
        
        for i in Cal_list:
            
            save_path = os.path.join(output_folder, "{}_ADMeanFused_Trans.tif".format(f'{i:05d}'))
            save_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}'))

            if os.path.exists(save_path):
                array_ =  DataIOTrans.DataIO.read_IMG(save_path)
            elif os.path.exists(save_path2):
                
                save_path = save_path2
                array_ =  DataIOTrans.DataIO.read_IMG(save_path)
            else:
                print(f'Error: {save_path} or {save_path2} not found.')
                break 
            
            w,s,h = array_.shape
            if np.sum(array_[:,:,0] < 20) / (w * s) > 0.8 :
                print('remove path = {},原因是纯黑'.format(save_path))
                os.remove(save_path)
            if np.sum(array_[:,:,0] == 255)/ (w * s) > 0.5:
                print('remove path = {},原因是纯白'.format(save_path))
                os.remove(save_path)

START_DATE: 2015-02-25, END_DATE: 2015-05-31
START_DATE: 2015-05-31, END_DATE: 2015-09-15
START_DATE: 2015-09-15, END_DATE: 2015-11-28
remove path = D:\Dataset_and_Demo\2015-09-15_to_2015-11-28\00376_ADMeanFused_Trans.tif,原因是纯黑
START_DATE: 2015-11-28, END_DATE: 2016-02-25
START_DATE: 2016-02-25, END_DATE: 2016-05-31
remove path = D:\Dataset_and_Demo\2016-02-25_to_2016-05-31\01511_ADMeanFused_Trans.tif,原因是纯黑
START_DATE: 2016-05-31, END_DATE: 2016-09-15
remove path = D:\Dataset_and_Demo\2016-05-31_to_2016-09-15\00029_ADMeanFused_Trans.tif,原因是纯黑
START_DATE: 2016-09-15, END_DATE: 2016-11-28
START_DATE: 2016-11-28, END_DATE: 2017-02-25
START_DATE: 2017-02-25, END_DATE: 2017-05-31
remove path = D:\Dataset_and_Demo\2017-02-25_to_2017-05-31\02789_ADMeanFused_Trans.tif,原因是纯黑
START_DATE: 2017-05-31, END_DATE: 2017-09-15
remove path = D:\Dataset_and_Demo\2017-05-31_to_2017-09-15\00888_ADMeanFused_Trans.tif,原因是纯黑
START_DATE: 2017-09-15, END_DATE: 2017-11-28
START_DATE: 2017-11-28, END_DATE: 2018-0

## 删除没有_Trans.tif的SAM文件

In [3]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        
        # 不存在任意则保留
        Cal_list = [i for i in Cal_list if not (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
        
        for i in Cal_list:
            output_path = os.path.join(sam_predict_folder, "{}_ADMeanFused_SAM.tif".format(f'{i:05d}'))
            if os.path.exists(output_path):
                print('remove path = {}'.format(output_path))
                # os.remove(output_path)

START_DATE: 2015-02-25, END_DATE: 2015-05-31
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00094_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00132_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00151_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00153_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00207_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00271_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00279_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00318_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00328_ADMeanFused_SAM.tif
remove path = D:\Dataset_and_Demo\Processed\2015-02-25_to_2015-05-31\00380_ADMeanFused_SAM.tif
remov

## 删除所有_trans

In [6]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        
        # 存在任意则保留
        Cal_list = [i for i in Cal_list if (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
        
        
        for i in Cal_list:
            
            save_path = os.path.join(output_folder, "{}_ADMeanFused_Trans.tif".format(f'{i:05d}'))
            save_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}'))

            if os.path.exists(save_path):
                pass
            elif os.path.exists(save_path2):
                save_path = save_path2

            os.remove(save_path)

START_DATE: 2015-02-25, END_DATE: 2015-05-31
START_DATE: 2015-05-31, END_DATE: 2015-09-15
START_DATE: 2015-09-15, END_DATE: 2015-11-28
START_DATE: 2015-11-28, END_DATE: 2016-02-25
START_DATE: 2016-02-25, END_DATE: 2016-05-31
START_DATE: 2016-05-31, END_DATE: 2016-09-15
START_DATE: 2016-09-15, END_DATE: 2016-11-28
START_DATE: 2016-11-28, END_DATE: 2017-02-25
START_DATE: 2017-02-25, END_DATE: 2017-05-31
START_DATE: 2017-05-31, END_DATE: 2017-09-15
START_DATE: 2017-09-15, END_DATE: 2017-11-28
START_DATE: 2017-11-28, END_DATE: 2018-02-25
START_DATE: 2018-02-25, END_DATE: 2018-05-31


2024-06-07 00:22:45 WARNING Sleeping 0.99 seconds before retry 1 of 5 for request: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json, after HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))
2024-06-07 00:22:51 WARNING Sleeping 1.77 seconds before retry 2 of 5 for request: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json, after ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))


START_DATE: 2018-05-31, END_DATE: 2018-09-15
START_DATE: 2018-09-15, END_DATE: 2018-11-28
START_DATE: 2018-11-28, END_DATE: 2019-02-25
START_DATE: 2019-02-25, END_DATE: 2019-05-31
START_DATE: 2019-05-31, END_DATE: 2019-09-15
START_DATE: 2019-09-15, END_DATE: 2019-11-28
START_DATE: 2019-11-28, END_DATE: 2020-02-25
START_DATE: 2020-02-25, END_DATE: 2020-05-31
START_DATE: 2020-05-31, END_DATE: 2020-09-15


## 检测cunzai1空值tif ,删除

## 新建_Trans.tif并形成SAM.tif

In [ ]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        Cal_list = [i for i in Cal_list if not (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
            
        for i in Cal_list:
            
            save_path = os.path.join(output_folder, "{}_ADMeanFused.tif".format(f'{i:05d}'))
            json_path = os.path.join(output_folder, "{}_ADMeanFused_AOI.json".format(f'{i:05d}'))
            save_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles.tif".format(f'{i:05d}'))
            json_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles_AOI.json".format(f'{i:05d}'))
            
            output_path = os.path.join(sam_predict_folder, "{}_ADMeanFused_SAM.tif".format(f'{i:05d}'))
            
            if os.path.exists(save_path):
                imagePath = DataIOTrans.DataIO.TransImage_Values(save_path, transFunc=DataIOTrans.DataTrans.MinMaxBoundaryScaler,
                                            bandSave=[0, 0, 0], scale=255)
            elif os.path.exists(save_path2):
                imagePath = DataIOTrans.DataIO.TransImage_Values(save_path2, transFunc=DataIOTrans.DataTrans.MinMaxBoundaryScaler,
                                            bandSave=[0, 0, 0], scale=255)
                json_path = json_path2
            else:
                print(f'Error: {save_path} or {save_path2} not found.')
                break 
            
            if os.path.exists(json_path):
                with open(json_path, 'r') as f:
                    AOI_Bound_Info = json.load(f)
            else:
                print(f'Error: {json_path} not found.')
                break
            
            sam.set_image(imagePath)


            boxes = [*AOI_Bound_Info['coordinates'][0][0], *AOI_Bound_Info['coordinates'][0][2]]
            
            if os.path.exists(output_path):
                os.remove(output_path)
                print(f"文件 {output_path} ----------------------------删除")
            else:
                print(f"文件 {output_path} 不存在。")
            sam.predict(boxes=boxes, point_crs="EPSG:4326", output=output_path, dtype="uint8")